<a href="https://colab.research.google.com/github/nick01as/Gazer/blob/main/Stargazing_Complete_Dataframe_w_AQI_and_LM_%3E_SQM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

# Haversince Distance Formula (https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546)

def dist(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

AQData = pd.read_csv("AQDataCities.csv")
AQData = AQData.drop(columns = ['Unnamed: 0'])
AQData = AQData.sort_values(by = ['Latitude','Longitude'])

SQMData = pd.read_csv("SQMDataCity.csv")
SQMData = SQMData.drop(columns = ['Unnamed: 0'])
cols = list(SQMData.columns.values)
SQMData = SQMData[cols[:4] + [cols[-1]] + cols[4:6]]

LM2021 = pd.read_csv("LM2021City.csv")
LM2020 = pd.read_csv("LM2020City.csv")
LM2019 = pd.read_csv("LM2019City.csv")
LM2018 = pd.read_csv("LM2018City.csv")
LM2017 = pd.read_csv("LM2017City.csv")
LM2016 = pd.read_csv("LM2016City.csv")
LM2015 = pd.read_csv("LM2015City.csv")

LM2021 = LM2021.drop(columns = ['Unnamed: 0'])
LM2020 = LM2020.drop(columns = ['Unnamed: 0'])
LM2019 = LM2019.drop(columns = ['Unnamed: 0'])
LM2018 = LM2018.drop(columns = ['Unnamed: 0'])
LM2017 = LM2017.drop(columns = ['Unnamed: 0'])
LM2016 = LM2016.drop(columns = ['Unnamed: 0'])
LM2015 = LM2015.drop(columns = ['Unnamed: 0'])

LM2021 = LM2021.sort_values(by = ['City','LocalDate'])
LM2020 = LM2020.sort_values(by = ['City','LocalDate'])
LM2019 = LM2019.sort_values(by = ['City','LocalDate'])
LM2018 = LM2018.sort_values(by = ['City','LocalDate'])
LM2017 = LM2017.sort_values(by = ['City','LocalDate'])
LM2016 = LM2016.sort_values(by = ['City','LocalDate'])
LM2015 = LM2015.sort_values(by = ['City','LocalDate'])

AllData = pd.DataFrame()

def search_LM (city, date):
  years = [LM2015,LM2016,LM2017,LM2018,LM2019,LM2020,LM2021]
  LMData = years[int(date[:4]) - 2015]
  low = 0
  high = len(LMData.index)-1

  while low < high:
    mid = low + (high - low)//2
    if (LMData.iloc[mid,3] < city):
      low = mid + 1
    else:
      high = mid
  
  if LMData.iloc[low,3] != city:
    return 0
  
  return LMData.iloc[idx, 1]

def search_AQ (lat, long, date):
  lowLB = 0
  highLB = len(AQData.index) 
  lowUB = 0
  highUB = len(AQData.index) 

  while lowLB < highLB:
    mid = lowLB + (highLB - lowLB)//2
    if AQData.iloc[mid,4] < lat - 1.5:
      lowLB = mid + 1;
    else:
      highLB = mid;

  while lowUB < highUB:
    mid = lowUB + (highUB - lowUB)//2
    if AQData.iloc[mid,4] <= lat + 1.5:
      lowUB = mid + 1;
    else:
      highUB = mid;
  
  lowInt = lowLB
  highInt = lowUB
  
  best = 2147483647
  city = ""
  country = ""
  AQ = 0

  for idx in range (lowLB, lowUB):
    dif = dist(float(lat),float(long),float(AQData.iloc[idx,4]),float(AQData.iloc[idx,5]))
    if dif < best:
      city = AQData.iloc[idx,1]
      country = AQData.iloc[idx,0]
      best = dif
      AQ = AQData.iloc[idx,3]

  return (AQ,search_LM(city,date))

SQMList = []
AQList = []
LMList = []
DateList = []
CityList = []

idx = 0
while idx < len(SQMData.index):
  print (idx)
  if int(SQMData.iloc[idx,0][:4]) <= 2021 and int(SQMData.iloc[idx,0][:4]) >= 2015:
    SQMloc = (SQMData.iloc[idx,5],SQMData.iloc[idx,6],SQMData.iloc[idx,0])
  else:
    idx = idx + 1
    continue

  AQResult = search_AQ(SQMloc[0],SQMloc[1],SQMloc[2])

  if AQResult[0] != 0 and AQResult[1] != 0:
    LMList.append(AQResult[1])
    AQList.append(AQResult[0])
    SQMList.append(SQMData.iloc[idx,2])
    DateList.append(SQMloc[1])
    CityList.append(SQMData.iloc[idx,4])
  idx = idx + 1

AllData['Date'] = DateList
AllData['City'] = CityList
AllData["SQM"] = SQMList
AllData["LM"] = LMList
AllData["AQ"] = AQList

AllData

AllData.to_csv('AllData.csv')
files.download('AllData.csv')




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

SQMData = pd.read_csv("SQMDataCity.csv")
SQMData = SQMData.drop(columns = ['Unnamed: 0'])


SQMData

,UT_Date,UT_Time,Brightness,Conditions,City
0,2023-05-14,21:45:00,19.70,30% cloud cover,Zvenigorod
1,2023-05-13,09:00:00,19.08,,San Jose
2,2023-05-12,22:15:00,19.73,,Zvenigorod
3,2023-05-11,19:40:00,19.00,20% cloud cover,Zvenigorod
4,2023-05-10,06:22:01,21.69,,Garden City
...,...,...,...,...,...
2407,2005-08-06,11:00:00,21.23,,West Grey
2408,2005-08-06,02:00:00,21.52,,Briançon
2409,2005-08-05,03:15:00,17.05,,Hialeah
2410,2005-07-31,01:55:00,19.60,,Loyalist


In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

SQMData = pd.read_csv("SQMData.csv")
SQMData
SQMData = SQMData.drop(columns = [' Type',' Moon_alt',' Moon_illum'])

idx = 0
while idx < len(SQMData.index):
  if SQMData.iloc[idx,3] != 'Up' and SQMData.iloc[idx,3] != 'up':
    SQMData = SQMData.drop(index = SQMData.index[idx])
  else:
    idx = idx + 1
  if (SQMData.iloc[idx,4] != '30% cloud cover' and SQMData.iloc[idx,4] != '20% cloud cover' and SQMData.iloc[idx,4] != '10% cloud cover' and SQMData.iloc[idx,4] != 'Clear' 
      and SQMData.iloc[idx,4] != ' ' and SQMData.iloc[idx,4] != 'Clear'):
    SQMData = SQMData.drop(index = SQMData.index[idx])
  else:
    idx = idx + 1
  
SQMData = SQMData.drop(columns = [' Zenith',' Site description'])


SQMData

# completeData = pd.DataFrame()

# for idx in range 






KeyError: ignored

In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

# Haversince Distance Formula (https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546)

def dist(lat1, long1, lat2, long2):
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

SQMData = pd.read_csv("SQMDataAll.csv")
SQMData = SQMData.drop(columns = ['Unnamed: 0'])

AQData = pd.read_csv('AQDataCities.csv')
AQData = AQData.drop(columns = ['Unnamed: 0'])

LM2021 = pd.read_csv("LM2021City.csv")
LM2020 = pd.read_csv("LM2020City.csv")
LM2019 = pd.read_csv("LM2019City.csv")
LM2018 = pd.read_csv("LM2018City.csv")
LM2017 = pd.read_csv("LM2017City.csv")
LM2016 = pd.read_csv("LM2016City.csv")
LM2015 = pd.read_csv("LM2015City.csv")

LM2021 = LM2021.drop(columns = ['Unnamed: 0'])
LM2020 = LM2020.drop(columns = ['Unnamed: 0'])
LM2019 = LM2019.drop(columns = ['Unnamed: 0'])
LM2018 = LM2018.drop(columns = ['Unnamed: 0'])
LM2017 = LM2017.drop(columns = ['Unnamed: 0'])
LM2016 = LM2016.drop(columns = ['Unnamed: 0'])
LM2015 = LM2015.drop(columns = ['Unnamed: 0'])

LM2021 = LM2021.sort_values(by = ['City','LocalDate'])
LM2020 = LM2020.sort_values(by = ['City','LocalDate'])
LM2019 = LM2019.sort_values(by = ['City','LocalDate'])
LM2018 = LM2018.sort_values(by = ['City','LocalDate'])
LM2017 = LM2017.sort_values(by = ['City','LocalDate'])
LM2016 = LM2016.sort_values(by = ['City','LocalDate'])
LM2015 = LM2015.sort_values(by = ['City','LocalDate'])

def search_city(city):
  low = 0
  high = len(CityCoor.index)-1

  while low < high:
    mid = low + (high-low)//2
    if CityCoor.iloc[mid,2] < city:
      low = mid + 1
    else:
      high = mid

  if CityCoor.iloc[low,2] == city:
    return True
  else:
    return False

allCities = []

for idx in range(len(AQData)):
  if search_city(AQData.iloc[idx,1]) == True:
    allCities.append(AQData.iloc[idx,1])

idx = 0
while idx < len(SQMData.index):
  if SQMData.iloc[idx,4] == '0' or SQMData.iloc[idx,5] == '0' or SQMData.iloc[idx,4] == ' ' or SQMData.iloc[idx,5] == ' ':
    SQMData = SQMData.drop(index = SQMData.index[idx])
  else:
    idx = idx + 1

# Import City Coordinates
CityCoor = pd.read_csv("worldcities.csv")
CityCoor = CityCoor.drop(columns = ['city_ascii','iso2','iso3','admin_name','capital','population','id'])

# Sort City Locations
CityCoor = CityCoor.sort_values(['lat','lng'])
cols = list(CityCoor.columns.values)
CityCoor = CityCoor[cols[1:3] + [cols[0]] + [cols[3]]]

# Find City

def find_city(lat,lng):
  # print (lat,lng)
  lowLB = 0
  highLB = len(CityCoor.index) 
  lowUB = 0
  highUB = len(CityCoor.index) 

  while lowLB < highLB:
    mid = lowLB + (highLB - lowLB)//2
    if CityCoor.iloc[mid,0] < lat - 0.12:
      lowLB = mid + 1;
    else:
      highLB = mid;

  while lowUB < highUB:
    mid = lowUB + (highUB - lowUB)//2
    if CityCoor.iloc[mid,0] <= lat + 0.12:
      lowUB = mid + 1;
    else:
      highUB = mid;
  
  lowInt = lowLB
  highInt = lowUB

  # print("Lower Bound Coordinate", CityCoor.iloc[lowLB,0], CityCoor.iloc[highLB,0])
  # print("Upper Bound Coordinate", CityCoor.iloc[lowUB,0], CityCoor.iloc[highUB,0])
  best = 2147483647
  city = ""
  country = ""

  for idx in range (lowLB, lowUB):
    dif = dist(lat,lng,CityCoor.iloc[idx,0],CityCoor.iloc[idx,1])
    if dif < best:
      city = CityCoor.iloc[idx,2]
      country = CityCoor.iloc[idx,3]
      best = dif

  return (city,country)


cityList = []

for idx in range(len(SQMData.index)):
  print (idx)
  locInfo = find_city(float(SQMData.iloc[idx,4]),float(SQMData.iloc[idx,5]))
  cityList.append(locInfo[0])

SQMData['City'] = cityList

SQMData.to_csv('SQMDataCity.csv')
files.download('SQMDataCity.csv')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import math
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

AQData = pd.read_csv("AQData2022.csv")
AQData = AQData.drop(columns = ['WHO Region','ISO3','PM2.5 (μg/m3)','NO2 (μg/m3)','PM25 temporal coverage (%)','PM10 temporal coverage (%)',
                                'NO2 temporal coverage (%)','Reference','Number and type of monitoring stations','Version of the database',
                                'Status','Unnamed: 15','Unnamed: 16'])
def change_name (city):
  new_name = ''
  for i in city:
    if i == '-' or i == '/':
      break
    else:
      new_name = new_name + i

  return new_name

idx = 0
while idx < len(AQData.index):
  print (idx)
  AQData.iloc[idx,1] = change_name(AQData.iloc[idx,1])
  if math.isnan(AQData.iloc[idx,3]):
    AQData = AQData.drop(index = AQData.index[idx])
  else:
    idx = idx + 1

AQData.to_csv('AQDataClean.csv')
files.download('AQDataClean.csv')

Streaming output truncated to the last 5000 lines.
17390
17391
17392
17393
17394
17395
17396
17397
17398
17399
17400
17401
17402
17403
17404
17405
17406
17407
17408
17409
17410
17411
17412
17413
17414
17415
17416
17417
17418
17418
17418
17418
17418
17418
17419
17420
17421
17422
17423
17423
17423
17423
17423
17423
17423
17423
17424
17425
17426
17427
17428
17429
17430
17431
17432
17433
17434
17435
17436
17437
17438
17439
17440
17441
17442
17443
17444
17445
17446
17447
17448
17449
17450
17451
17452
17453
17454
17455
17456
17457
17458
17459
17460
17461
17462
17463
17463
17464
17465
17466
17467
17468
17469
17470
17471
17472
17473
17474
17475
17476
17477
17478
17479
17480
17481
17482
17483
17484
17485
17486
17487
17488
17489
17490
17491
17492
17493
17494
17495
17496
17497
17498
17499
17500
17501
17502
17503
17504
17505
17506
17507
17508
17509
17510
17511
17512
17512
17513
17514
17515
17515
17516
17517
17518
17519
17520
17521
17522
17523
17524
17525
17526
17527
17528
17529
17530
17531
17532
1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import opendatasets as od
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import radians, cos, sin, asin, sqrt
from google.colab import files

AQData = pd.read_csv("AQDataClean.csv")
AQData = AQData.drop(columns = ['Unnamed: 0'])

# Import City Coordinates
CityCoor = pd.read_csv("worldcities.csv")
CityCoor = CityCoor.drop(columns = ['city_ascii','iso2','iso3','admin_name','capital','population','id'])

# Sort City Locations
CityCoor = CityCoor.sort_values(['city'])
cols = list(CityCoor.columns.values)
CityCoor = CityCoor[cols[1:3] + [cols[0]] + [cols[3]]]

def change_name (city):
  new_name = ''
  for i in city:
    if i == '-' or i == '/' or i == '(':
      break
    else:
      new_name = new_name + i

  return new_name

def search_city(city):
  low = 0
  high = len(CityCoor.index)-1

  while low < high:
    mid = low + (high-low)//2
    if CityCoor.iloc[mid,2] < city:
      low = mid + 1
    else:
      high = mid

  if CityCoor.iloc[low,2] == city:
    return (True,CityCoor.iloc[low,0],CityCoor.iloc[low,1])
  else:
    return (False,0,0)

lat = []
lng = []

idx = 0
while idx < len(AQData.index):
  print (AQData.iloc[idx,1])
  AQData.iloc[idx,1] = change_name(AQData.iloc[idx,1])
  result = search_city(AQData.iloc[idx,1])
  if result[0] == False:
    AQData = AQData.drop(index = AQData.index[idx])
  else:
    lat.append(result[1])
    lng.append(result[2])
    idx = idx + 1

AQData['Latitude'] = lat
AQData['Longitude'] = lng
AQData.to_csv('AQDataCities.csv')
files.download('AQDataCities.csv')

Streaming output truncated to the last 5000 lines.
Valthermond
Valthermond
Veldhoven
Veldhoven
Veldhoven
Veldhoven
Veldhoven
Veldhoven
Veldhoven
Velsen
Velsen
Velsen
Velsen
Velsen
Vlaardingen
Vlaardingen
Vlaardingen
Vlaardingen
Vlaardingen
Vlaardingen
Vlaardingen
Vredepeel
Vredepeel
Vredepeel
Vredepeel
Vredepeel
Vredepeel
Vredepeel
Vredepeel
Wekerom
Wekerom
Wekerom
Wekerom
Wekerom
Wekerom
Wekerom
Wekerom
Westmaas
Westmaas
Westmaas
Westmaas
Westmaas
Westmaas
Westmaas
Westmaas
Westzaan
Westzaan
Westzaan
Westzaan
Westzaan
Westzaan
Westzaan
Wieringerwerf
Wieringerwerf
Wieringerwerf
Wieringerwerf
Wieringerwerf
Wieringerwerf
Wieringerwerf
Wieringerwerf
Wijk Aan Zee
Wijk Aan Zee
Wijk Aan Zee
Wijk Aan Zee
Wijk Aan Zee
Wijk Aan Zee
Wijk Aan Zee
Wijnandsrade
Wijnandsrade
Wijnandsrade
Wijnandsrade
Wijnandsrade
Wijnandsrade
Wijnandsrade
Wijnandsrade
Zaandam
Zaandam
Zaandam
Zaandam
Zaandam
Zaandam
Zaandam
Zaandam
Zegveld
Zegveld
Zegveld
Zegveld
Zegveld
Zegveld
Zegveld
Zegveld
Aalesund
Alesund
Alesu

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>